In [1]:
#imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time
import datetime
from nba_api.stats.library.http import NBAStatsHTTP
import joblib

# Set the timeout
NBAStatsHTTP.timeout = 120  # Set to 30 seconds; adjust as needed

In [2]:
def player_name(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_Name = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_NAME'].iloc[0]
    return player_Name
# print(player_name(1629029))
def player_id(player_Name):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_ID = players_data.loc[players_data['PLAYER_NAME'] == player_Name, 'PLAYER_ID'].iloc[0]
    return player_ID
# Example, should return 1629029
# player_id('Luka Dončić')

In [3]:
def team_id(team_abbrev):
    teams = teamgamelogs.TeamGameLogs()
    teams_data = teams.get_data_frames()[0]
    team_id = teams_data.loc[teams_data['TEAM_ABBREVIATION'] == team_abbrev, 'TEAM_ID'].iloc[0]
    return team_id
# Example usage, should return: 1610612747
# print(team_id('LAL'))
def team_abbrev(team_ID):
    teams = teamdetails.TeamDetails(team_id= team_ID)
    teams_data = teams.get_data_frames()[0]
    team_abbrev = teams_data.loc[teams_data['TEAM_ID'] == team_ID, 'ABBREVIATION'].iloc[0]
    return team_abbrev
# team_abbrev(1610612747)

In [4]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    exc = gamelog.get_data_frames()[0]
    return gamelog.get_data_frames()[0]
# player_gamelog('Jaren Jackson Jr.')


In [5]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

# playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [6]:
def age(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    players_data.to_excel('Players_Data.xlsx')
    player_age = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'AGE'].iloc[0]
    return player_age
# age(player_id('Luka Dončić'))

In [7]:
def height(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_height = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_HEIGHT_INCHES'].iloc[0]
    return player_height
# Example, should return 83(in)
# height(player_id('Nikola Jokić'))

In [8]:
def weight(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_weight = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_WEIGHT'].iloc[0]
    return player_weight
# weight(player_id('Luka Dončić'))

In [9]:
def encode_positions(position):
    roles = {'Primary_Guard': 0, 'Primary_Forward': 0, 'Primary_Center': 0, 'Is_Guard': 0, 'Is_Forward': 0, 'Is_Center': 0}
    primary = position.split('-')[0]
    if 'G' == primary:
        roles['Primary_Guard'] = 1
    elif 'F' == primary:
        roles['Primary_Forward'] = 1
    elif 'C' == primary:
        roles['Primary_Center'] = 1
    if 'G' in position:
        roles['Is_Guard'] = 1
    if 'F' in position:
        roles['Is_Forward'] = 1
    if 'C' in position:
        roles['Is_Center'] = 1
    return pd.Series(roles)
# print(encode_positions('C-G'))
def position(player_ID):
    player_Name = player_name(player_ID)
    firstName = player_Name.split(" ")[0]
    lastName = player_Name.split(" ")[1]
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    position = df.loc[(df['PLAYER_LAST_NAME'] == lastName) & (df['PLAYER_FIRST_NAME'] == firstName), 'POSITION'].iloc[0]
    return position 
# position(2544), "LeBron, F"

In [10]:
def player_team(player_name):
    firstName = player_name.split(" ")[0]
    lastName = player_name.split(" ")[1]
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    team_ID = df.loc[(df['PLAYER_LAST_NAME'] == lastName) & (df['PLAYER_FIRST_NAME'] == firstName), 'TEAM_ID'].iloc[0]
    team = team_abbrev(team_ID)
    return team
# player_team('Jalen Brunson')

In [11]:
# Will add a Home_Away column to the gamelog so that the ML model can use that as a variable
def homeOrAway(player_gamelog):
    player_gamelog['Home_Away'] = player_gamelog['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return player_gamelog
# kingStats = player_gamelog('LeBron James')
# homeOrAway

In [12]:
def extract_team(matchup):
     # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[0]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[0]
    else:
        return "Invalid Matchup Format"
# print(extract_team('LAL @ NYK'))

def extract_opponent(matchup):
    # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[1]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[1]
    else:
        return "Invalid Matchup Format"
# extract_opponent('LAL @ NYK')

In [13]:
def againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    awaySearchString = home_team_abbrev + ' vs. ' + opp_team_abbrev
    homeSearchString = home_team_abbrev + ' @ ' + opp_team_abbrev
    newGameLog = player_gamelog.loc[player_gamelog.MATCHUP.isin([homeSearchString, awaySearchString])]
    return newGameLog

# Test
# kingStats = player_gamelog('LeBron James')
# print(againstThisTeam(kingStats, 'LAL', 'ATL'))
def boxStatsAgainstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    new_log = againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev)
    new_log = new_log.loc[:, ['GAME_DATE', 'MATCHUP', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    return new_log
# test
# dfoxStats = player_gamelog("De'Aaron Fox")
# boxStatsAgainstThisTeam(dfoxStats, 'SAC', 'MEM')

In [14]:
def getOFFefficiency(team_ID):
    offenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = offenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_OFF_RATING'].iloc[0]
    return defEfficiency
# print(getOFFefficiency(team_id('CLE')))
def getPPG(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['PointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# getPPG(team_id('LAL'))

In [15]:
# Note: the numbers in the def efficiency do not match the numbers on the official website, yet they are in the correct rankings, (i.e OKC's def efficiency is #1 in this data, and #1 on the official data, however they have differing def efficiency numbers, one could be 102.9 the other 103.1)
def getDEFefficiency(team_ID):
    defenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = defenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_DEF_RATING'].iloc[0]
    return defEfficiency
# print(getDEFefficiency(team_id('LAL')))
    
def getPPG_allowed(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    df_teamstats.to_excel('teamstats_.xlsx')
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['OppPointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# Example Usage, Output: 113.70588235294117
# getPPG_allowed(team_id('LAL'))


In [16]:
# NOTE!!!!!!!!!!!! THIS SHOULD ONLY BE USED ONCE TO CREATE A NEW CSV FOR A PLAYER. ONCE A CSV HAS BEEN CREATED, USE THE updateStats METHOD TO UPDATE THEIR MOST CURRENT GAME.

def importantStats(player_name):
    player_gameLog = player_gamelog(player_name)
    player_gameLog = player_gameLog.iloc[::-1].reset_index(drop=True)
    window_size = 5
    stats_to_average = ['PTS', 'FGM', 'FGA', 'STL', 'BLK', 'TOV', 'FG_PCT', 'MIN', 'PLUS_MINUS']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = player_gameLog[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
    player_gameLog['AVG_PTS'] = player_gameLog['PTS'].expanding().mean().shift(1)

    # Extract opponent team abbreviation
    player_gameLog['TEAM_ABBREVIATION'] = player_gameLog['MATCHUP'].apply(extract_team)
    player_gameLog['TEAM_ID'] = player_gameLog['TEAM_ABBREVIATION'].apply(team_id)
    player_gameLog['OPP_TEAM'] = player_gameLog['MATCHUP'].apply(extract_opponent)
    player_gameLog['OPP_TEAMID'] = player_gameLog['OPP_TEAM'].apply(team_id)
    # Fetch the PPG allowed by the opponent team
    player_gameLog['OPP_PPG_ALLOWED'] = player_gameLog['OPP_TEAMID'].apply(getPPG_allowed)
    player_gameLog['OPP_DEF_EFFICIENCY'] = player_gameLog['OPP_TEAMID'].apply(getDEFefficiency)
    player_gameLog['PPG'] = player_gameLog['TEAM_ID'].apply(getPPG)
    player_gameLog['OFF_EFFICIENCY'] = player_gameLog['TEAM_ID'].apply(getOFFefficiency)
    player_gameLog['POSITION'] = player_gameLog['Player_ID'].apply(position)
    # Need to encode positions here
    player_gameLog['AGE'] = player_gameLog['Player_ID'].apply(age)
    player_gameLog['HEIGHT'] = player_gameLog['Player_ID'].apply(height)
    player_gameLog['WEIGHT'] = player_gameLog['Player_ID'].apply(weight)

    homeOrAway(player_gameLog)
    return player_gameLog
# Example Usage, runtime ~1m 18s
# Missing Jaren Jackson Jr. and Michael Porter Jr. <- the "Jr." is the issue
players_list = [
    "Alperen Sengun", "Anfernee Simons", "Anthony Davis", "Anthony Edwards", "Austin Reaves", "Brandon Miller", "Cade Cunningham", "Cameron Johnson", "Coby White", "Damian Lillard", "Darius Garland", "De'Aaron Fox", "DeMar Rozan", "Devin Booker", "Domantas Sabonis", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kyrie Irving", "Lauri Markkanen", "LeBron James", "Mikal Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "RJ Barrett", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine"
]


In [17]:
# USE THIS METHOD ONCE A CSV HAS BEEN CREATED FOR A PLAYER
def updateStats(player_csv, player_name):
    df = pd.read_csv(player_csv)
    df_new = importantStats(player_name)
    last_row = df_new.tail(1)
    df = pd.concat([df, last_row], ignore_index=True)
    folder_name = 'players csvs'
    file_path = os.path.join(folder_name, player_name + '.csv')
    df.to_csv(file_path)
    return df
updateStats('players csvs/Anfernee Simons.csv', 'Anfernee Simons')

,Unnamed: 0,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,...,OPP_TEAMID,OPP_PPG_ALLOWED,OPP_DEF_EFFICIENCY,PPG,OFF_EFFICIENCY,POSITION,AGE,HEIGHT,WEIGHT,Home_Away
0,0.0,22024,1629014,22400072,"OCT 23, 2024",POR vs. GSW,L,33,5,12,...,1610612744,111.1,107.8,108.1,105.4,G,25.0,75,200,1
1,1.0,22024,1629014,22400086,"OCT 25, 2024",POR vs. NOP,L,36,6,20,...,1610612740,117.1,114.7,108.1,105.4,G,25.0,75,200,1
2,2.0,22024,1629014,22400099,"OCT 27, 2024",POR vs. NOP,W,33,8,17,...,1610612740,117.1,114.7,108.1,105.4,G,25.0,75,200,1
3,3.0,22024,1629014,22400112,"OCT 28, 2024",POR @ SAC,L,32,4,12,...,1610612758,113.8,110.6,108.1,105.4,G,25.0,75,200,0
4,4.0,22024,1629014,22400127,"OCT 30, 2024",POR @ LAC,W,36,10,24,...,1610612746,107.9,107.0,108.1,105.4,G,25.0,75,200,0
5,5.0,22024,1629014,22400140,"NOV 01, 2024",POR vs. OKC,L,28,1,8,...,1610612760,103.0,100.7,108.1,105.4,G,25.0,75,200,1
6,6.0,22024,1629014,22400149,"NOV 02, 2024",POR @ PHX,L,41,8,25,...,1610612756,113.9,113.4,108.1,105.4,G,25.0,75,200,0
7,7.0,22024,1629014,22400164,"NOV 04, 2024",POR @ NOP,W,34,10,17,...,1610612740,117.1,114.7,108.1,105.4,G,25.0,75,200,0
8,8.0,22024,1629014,22400183,"NOV 07, 2024",POR @ SAS,L,33,7,16,...,1610612759,111.5,109.1,108.1,105.4,G,25.0,75,200,0
9,9.0,22024,1629014,22400193,"NOV 08, 2024",POR @ MIN,L,27,6,16,...,1610612750,107.7,107.2,108.1,105.4,G,25.0,75,200,0


In [18]:
# This assumes the player_name you are passing in, you have already created a CSV
def createModel(player_name):
    df = pd.read_csv('players csvs/Anfernee Simons.csv')
    df = df.drop(df.index[0]).reset_index(drop=True)
    df = df.iloc[7:].reset_index(drop=True)
    df.dropna()
    # Assuming you need to calculate average points again
    average_points = df['PTS'].mean()
    df['target'] = (df['PTS'] > average_points).astype(int)
    categorical_features = ['MATCHUP', 'Home_Away']
    one_hot = OneHotEncoder()

    # Define the pipeline
    classifier = Pipeline([
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')),
        ('classifier', LogisticRegression(max_iter=1000))
    ])

    # Assuming the features remain the same or adjust as necessary
    X = df[['MATCHUP', 'OPP_DEF_EFFICIENCY', 'OPP_PPG_ALLOWED', 'PPG', 'OFF_EFFICIENCY', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_BLK', 'MA_TOV', 'MA_FG_PCT', 'MA_MIN', 'MA_PLUS_MINUS', 'Home_Away']]  # Features
    y = df['target']                # Target variable

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    # Print results
    print(f"{player_name} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    
    # Ensure the output directory exists
    output_folder = 'players models'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the new model
    joblib.dump(classifier, f'{output_folder}/{player_name}.pkl')

players_list = [
    "Anfernee Simons", "Anthony Davis", "Anthony Edwards", "Austin Reaves", "Brandon Miller", "Cade Cunningham", "Cameron Johnson", "Coby White", "Damian Lillard", "Darius Garland", "De'Aaron Fox", "DeMar Rozan", "Devin Booker", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kyrie Irving", "Lauri Markkanen", "Mikal Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "RJ Barrett", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine"
]
# Every time you create a fresh new model, delete the name from the list
createModel('Anfernee Simons')

Anfernee Simons - Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.57      0.80      0.67         5

    accuracy                           0.50         8
   macro avg       0.29      0.40      0.33         8
weighted avg       0.36      0.50      0.42         8



In [19]:
sabonis_stats = pd.read_csv('players csvs/LeBron James.csv')
print(sabonis_stats.iloc[-1][['MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_BLK', 'MA_TOV', 'MA_FG_PCT', 'MA_MIN', 'MA_PLUS_MINUS']])

MA_PTS             26.0
MA_FGM             10.2
MA_FGA             18.0
MA_STL              0.4
MA_BLK              0.2
MA_TOV              2.6
MA_FG_PCT        0.5558
MA_MIN             34.2
MA_PLUS_MINUS      -5.6
Name: 32, dtype: object


In [20]:
# Sample data for a new game
df = pd.read_csv('players csvs/LeBron James.csv')
last_row = df.iloc[-1:]
new_game = {
    'MATCHUP': ['LAL @ DAL'],
    'OPP_DEF_EFFICIENCY': [getDEFefficiency(team_id('DAL'))],
    'OPP_PPG_ALLOWED': [getPPG_allowed(team_id('DAL'))],
    'PPG': [getPPG(team_id('LAL'))],
    'OFF_EFFICIENCY': [getOFFefficiency(team_id('LAL'))],
    'MA_PTS': [last_row['MA_PTS'].values[0]],
    'MA_FGM': [last_row['MA_FGM'].values[0]],
    'MA_FGA': [last_row['MA_FGA'].values[0]],
    'MA_STL': [last_row['MA_STL'].values[0]],
    'MA_BLK': [last_row['MA_BLK'].values[0]],
    'MA_TOV': [last_row['MA_TOV'].values[0]],
    'MA_FG_PCT': [last_row['MA_FG_PCT'].values[0]],
    'MA_MIN': [last_row['MA_MIN'].values[0]],
    'Home_Away': ['Away']
}
# Create a DataFrame
df_new_game = pd.DataFrame(new_game)

classifier = joblib.load('players models/lebron.pkl')

predicted_target = classifier.predict(df_new_game)

# Output the prediction
print("Predicted Target:", predicted_target)

probabilities = classifier.predict_proba(df_new_game)

# Since it's likely a binary classification, probabilities[:, 1] will give you the probability of class 1
predicted_probabilities = probabilities[:, 1]

# Output the probabilities
print("Average Points(line):", last_row['AVG_PTS'].iloc[0])
print("Probability of Class 1(over) (Predicted Target):", predicted_probabilities)

FileNotFoundError: [Errno 2] No such file or directory: 'players models/lebron.pkl'